# Tags y Resource Monitors para Control Granular de Costos — Versión para **Snowflake Notebooks**
**Fecha:** 2026-01-05
**Autor:** Juan Sierra
**Tags:** #snowflake #snowpark #python #sql #cost #tags #resource-monitors #governance

Este cuaderno está optimizado para ejecutarse dentro de **Snowflake Notebooks** usando **Snowpark**.
Cuando se ejecute fuera de Snowflake Notebooks (por ejemplo, en tu Jupyter local), hará *fallback* a una conexión estándar con credenciales.

> **Objetivo:** Implementar buenas prácticas de etiquetado (tags) y resource monitors para tener un control granular de los costos en Snowflake.

---

## Tabla de Contenidos
1. Parámetros y Conexión
2. Fundamentos de Tags en Snowflake
3. Creación y Gestión de Tags
4. Asignación de Tags a Objetos
5. Consultas de Costos por Tags
6. Resource Monitors
7. Alertas y Notificaciones
8. Buenas Prácticas


## 1. Parámetros

Ajusta estos valores predeterminados para tu entorno y ventana de análisis.


In [ ]:
# 1.1 Global parameters
M_DAYS = 30
TAG_DATABASE = "GOVERNANCE"
TAG_SCHEMA = "TAGS"
COST_CENTER_TAG = "COST_CENTER"
PROJECT_TAG = "PROJECT"
ENVIRONMENT_TAG = "ENVIRONMENT"
CREDIT_QUOTA_MONTHLY = 1000  # Créditos mensuales por defecto


## 2. Conexión y contexto de sesión

Esta celda usa `get_active_session()` dentro de **Snowflake Notebooks**. Si ejecutas localmente, hace fallback para crear una sesión.
También establece **ROLE**, **WAREHOUSE**, **DATABASE** y **SCHEMA** para obtener resultados consistentes.


In [ ]:
# 2.1 Session and context
from __future__ import annotations
import os, getpass
import pandas as pd
try:
    from snowflake.snowpark.context import get_active_session
    from snowflake.snowpark import Session
    session = get_active_session()
    IN_SF_NB = True
except Exception:
    from snowflake.snowpark import Session
    IN_SF_NB = False

ROLE      = os.getenv("SNOWFLAKE_ROLE",      "ACCOUNTADMIN")
WAREHOUSE = os.getenv("SNOWFLAKE_WAREHOUSE", "COMPUTE_WH")
DATABASE  = os.getenv("SNOWFLAKE_DATABASE",  "SNOWFLAKE")
SCHEMA    = os.getenv("SNOWFLAKE_SCHEMA",    "ACCOUNT_USAGE")

def get_session():
    global session
    if IN_SF_NB:
        return session
    conn = {
        "account":   os.getenv("SNOWFLAKE_ACCOUNT", "your_account"),
        "user":      os.getenv("SNOWFLAKE_USER", "your_user"),
        "password":  os.getenv("SNOWFLAKE_PASSWORD") or getpass.getpass("Snowflake password: "),
        "role":      ROLE, "warehouse": WAREHOUSE, "database": DATABASE, "schema": SCHEMA
    }
    session = Session.builder.configs(conn).create()
    return session

def run_sql_df(sql: str):
    s = get_session()
    return s.sql(sql).to_pandas()

def run_sql(sql: str):
    """Ejecuta SQL sin retornar DataFrame (para DDL/DML)"""
    s = get_session()
    return s.sql(sql).collect()

_ = run_sql_df(f"""
    USE ROLE {ROLE};
    USE WAREHOUSE {WAREHOUSE};
    USE DATABASE {DATABASE};
    USE SCHEMA {SCHEMA};
""")
run_sql_df("SELECT CURRENT_ROLE() ROLE, CURRENT_WAREHOUSE() WH, CURRENT_DATABASE() DB, CURRENT_SCHEMA() SCH;")


---
## 3. Fundamentos de Tags en Snowflake

Los **Tags** en Snowflake son objetos de esquema que permiten asignar metadatos a otros objetos. Son fundamentales para:

- **Asignación de costos:** Atribuir costos a centros de costo, proyectos o departamentos
- **Clasificación de datos:** Categorizar datos sensibles o PII
- **Seguridad:** Implementar políticas de acceso basadas en tags
- **Reportes:** Generar informes de uso granulares

### Objetos que soportan Tags:
- Warehouses
- Databases / Schemas
- Tables / Views
- Columns
- Users / Roles


### 3.1 Verificar Tags existentes en la cuenta


In [ ]:
# 3.1.1 Listar todos los tags definidos en la cuenta
sql = """
SELECT
    tag_database,
    tag_schema,
    tag_name,
    tag_owner,
    allowed_values,
    created,
    last_altered
FROM SNOWFLAKE.ACCOUNT_USAGE.TAGS
WHERE deleted IS NULL
ORDER BY tag_database, tag_schema, tag_name;
"""
run_sql_df(sql)


In [ ]:
# 3.1.2 Ver referencias de tags (qué objetos tienen tags asignados)
sql = """
SELECT
    tag_name,
    tag_value,
    object_database,
    object_schema,
    object_name,
    domain AS object_type,
    column_name
FROM SNOWFLAKE.ACCOUNT_USAGE.TAG_REFERENCES
WHERE deleted IS NULL
ORDER BY tag_name, object_database, object_name;
"""
run_sql_df(sql)


---
## 4. Creación y Gestión de Tags

### Buenas Prácticas para Estrategia de Etiquetado:

1. **Base de datos centralizada:** Crear una base de datos dedicada para governance
2. **Convenciones de nombres:** Usar prefijos claros (ej: `COST_`, `ENV_`, `DEPT_`)
3. **Valores permitidos:** Definir valores específicos para evitar inconsistencias
4. **Documentación:** Mantener un diccionario de tags actualizado


### 4.1 Crear base de datos y schema para Tags (Governance)


In [ ]:
# 4.1.1 Crear estructura de governance (ejecutar solo una vez)
sql_governance = f"""
-- Crear base de datos para governance
CREATE DATABASE IF NOT EXISTS {TAG_DATABASE}
    COMMENT = 'Base de datos centralizada para tags, políticas y governance';

-- Crear schema para tags
CREATE SCHEMA IF NOT EXISTS {TAG_DATABASE}.{TAG_SCHEMA}
    COMMENT = 'Schema para definición de tags de costos y governance';
"""
print("Ejecutar manualmente si es necesario:")
print(sql_governance)


### 4.2 Crear Tags para Control de Costos


In [ ]:
# 4.2.1 Definiciones de Tags recomendados para costos
sql_create_tags = f"""
-- Tag para Centro de Costo
CREATE TAG IF NOT EXISTS {TAG_DATABASE}.{TAG_SCHEMA}.{COST_CENTER_TAG}
    ALLOWED_VALUES 'MARKETING', 'ENGINEERING', 'DATA_SCIENCE', 'FINANCE', 'OPERATIONS', 'HR'
    COMMENT = 'Centro de costo para asignación de gastos';

-- Tag para Proyecto
CREATE TAG IF NOT EXISTS {TAG_DATABASE}.{TAG_SCHEMA}.{PROJECT_TAG}
    COMMENT = 'Nombre del proyecto o iniciativa';

-- Tag para Ambiente
CREATE TAG IF NOT EXISTS {TAG_DATABASE}.{TAG_SCHEMA}.{ENVIRONMENT_TAG}
    ALLOWED_VALUES 'DEV', 'QA', 'STAGING', 'PROD'
    COMMENT = 'Ambiente de ejecución';

-- Tag para Propietario
CREATE TAG IF NOT EXISTS {TAG_DATABASE}.{TAG_SCHEMA}.OWNER
    COMMENT = 'Propietario o responsable del recurso';

-- Tag para Fecha de Expiración
CREATE TAG IF NOT EXISTS {TAG_DATABASE}.{TAG_SCHEMA}.EXPIRATION_DATE
    COMMENT = 'Fecha de expiración del recurso temporal';

-- Tag para Criticidad
CREATE TAG IF NOT EXISTS {TAG_DATABASE}.{TAG_SCHEMA}.CRITICALITY
    ALLOWED_VALUES 'LOW', 'MEDIUM', 'HIGH', 'CRITICAL'
    COMMENT = 'Nivel de criticidad del recurso';
"""
print("Ejecutar para crear los tags:")
print(sql_create_tags)


---
## 5. Asignación de Tags a Objetos

### Ejemplos de cómo asignar tags a diferentes tipos de objetos


### 5.1 Asignar Tags a Warehouses


In [ ]:
# 5.1.1 Ejemplo: Asignar tags a un warehouse
sql_tag_warehouse = f"""
-- Asignar múltiples tags a un warehouse
ALTER WAREHOUSE <WAREHOUSE_NAME> SET TAG
    {TAG_DATABASE}.{TAG_SCHEMA}.{COST_CENTER_TAG} = 'ENGINEERING',
    {TAG_DATABASE}.{TAG_SCHEMA}.{ENVIRONMENT_TAG} = 'PROD',
    {TAG_DATABASE}.{TAG_SCHEMA}.{PROJECT_TAG} = 'DATA_PLATFORM';
"""
print("Ejemplo de asignación de tags a warehouse:")
print(sql_tag_warehouse)


In [ ]:
# 5.1.2 Listar warehouses y sus tags
sql = f"""
SELECT
    tr.object_name AS warehouse_name,
    tr.tag_name,
    tr.tag_value,
    w.warehouse_size,
    w.auto_suspend,
    w.auto_resume
FROM SNOWFLAKE.ACCOUNT_USAGE.TAG_REFERENCES tr
JOIN SNOWFLAKE.ACCOUNT_USAGE.WAREHOUSES w
    ON tr.object_name = w.warehouse_name
WHERE tr.domain = 'WAREHOUSE'
    AND tr.deleted IS NULL
    AND w.deleted IS NULL
ORDER BY tr.object_name, tr.tag_name;
"""
run_sql_df(sql)


### 5.2 Asignar Tags a Databases y Schemas


In [ ]:
# 5.2.1 Ejemplo: Asignar tags a database y schema
sql_tag_db = f"""
-- Asignar tag a base de datos
ALTER DATABASE <DATABASE_NAME> SET TAG
    {TAG_DATABASE}.{TAG_SCHEMA}.{COST_CENTER_TAG} = 'DATA_SCIENCE',
    {TAG_DATABASE}.{TAG_SCHEMA}.{ENVIRONMENT_TAG} = 'PROD';

-- Asignar tag a schema
ALTER SCHEMA <DATABASE_NAME>.<SCHEMA_NAME> SET TAG
    {TAG_DATABASE}.{TAG_SCHEMA}.{PROJECT_TAG} = 'ML_PIPELINE';
"""
print("Ejemplo de asignación de tags a database/schema:")
print(sql_tag_db)


### 5.3 Asignar Tags a Usuarios y Roles


In [ ]:
# 5.3.1 Ejemplo: Asignar tags a usuarios
sql_tag_user = f"""
-- Asignar tag a usuario (útil para atribución de costos por usuario)
ALTER USER <USERNAME> SET TAG
    {TAG_DATABASE}.{TAG_SCHEMA}.{COST_CENTER_TAG} = 'MARKETING',
    {TAG_DATABASE}.{TAG_SCHEMA}.OWNER = 'team_lead@company.com';
"""
print("Ejemplo de asignación de tags a usuarios:")
print(sql_tag_user)


---
## 6. Consultas de Costos por Tags

Queries para analizar y reportar costos basados en tags asignados.


### 6.1 Consumo de créditos por Centro de Costo


In [ ]:
# 6.1.1 Créditos consumidos por tag COST_CENTER (últimos m días)
sql = f"""
WITH warehouse_tags AS (
    SELECT
        object_name AS warehouse_name,
        tag_value AS cost_center
    FROM SNOWFLAKE.ACCOUNT_USAGE.TAG_REFERENCES
    WHERE domain = 'WAREHOUSE'
        AND tag_name = '{COST_CENTER_TAG}'
        AND deleted IS NULL
)
SELECT
    COALESCE(wt.cost_center, 'SIN_TAG') AS cost_center,
    wmh.warehouse_name,
    SUM(wmh.credits_used) AS total_credits,
    SUM(wmh.credits_used_compute) AS compute_credits,
    SUM(wmh.credits_used_cloud_services) AS cloud_services_credits
FROM SNOWFLAKE.ACCOUNT_USAGE.WAREHOUSE_METERING_HISTORY wmh
LEFT JOIN warehouse_tags wt
    ON wmh.warehouse_name = wt.warehouse_name
WHERE wmh.start_time >= DATEADD(day, -{M_DAYS}, CURRENT_TIMESTAMP())
GROUP BY 1, 2
ORDER BY total_credits DESC;
"""
run_sql_df(sql)


In [ ]:
# 6.1.2 Resumen total por Centro de Costo
sql = f"""
WITH warehouse_tags AS (
    SELECT
        object_name AS warehouse_name,
        tag_value AS cost_center
    FROM SNOWFLAKE.ACCOUNT_USAGE.TAG_REFERENCES
    WHERE domain = 'WAREHOUSE'
        AND tag_name = '{COST_CENTER_TAG}'
        AND deleted IS NULL
)
SELECT
    COALESCE(wt.cost_center, 'SIN_TAG') AS cost_center,
    COUNT(DISTINCT wmh.warehouse_name) AS num_warehouses,
    SUM(wmh.credits_used) AS total_credits,
    ROUND(100 * RATIO_TO_REPORT(SUM(wmh.credits_used)) OVER (), 2) AS pct_of_total
FROM SNOWFLAKE.ACCOUNT_USAGE.WAREHOUSE_METERING_HISTORY wmh
LEFT JOIN warehouse_tags wt
    ON wmh.warehouse_name = wt.warehouse_name
WHERE wmh.start_time >= DATEADD(day, -{M_DAYS}, CURRENT_TIMESTAMP())
GROUP BY 1
ORDER BY total_credits DESC;
"""
run_sql_df(sql)


### 6.2 Consumo de créditos por Ambiente


In [ ]:
# 6.2.1 Créditos por ambiente (DEV, QA, STAGING, PROD)
sql = f"""
WITH warehouse_tags AS (
    SELECT
        object_name AS warehouse_name,
        tag_value AS environment
    FROM SNOWFLAKE.ACCOUNT_USAGE.TAG_REFERENCES
    WHERE domain = 'WAREHOUSE'
        AND tag_name = '{ENVIRONMENT_TAG}'
        AND deleted IS NULL
)
SELECT
    COALESCE(wt.environment, 'SIN_TAG') AS environment,
    SUM(wmh.credits_used) AS total_credits,
    ROUND(100 * RATIO_TO_REPORT(SUM(wmh.credits_used)) OVER (), 2) AS pct_of_total
FROM SNOWFLAKE.ACCOUNT_USAGE.WAREHOUSE_METERING_HISTORY wmh
LEFT JOIN warehouse_tags wt
    ON wmh.warehouse_name = wt.warehouse_name
WHERE wmh.start_time >= DATEADD(day, -{M_DAYS}, CURRENT_TIMESTAMP())
GROUP BY 1
ORDER BY total_credits DESC;
"""
run_sql_df(sql)


### 6.3 Tendencia de costos por tag (diario)


In [ ]:
# 6.3.1 Tendencia diaria de créditos por Centro de Costo
sql = f"""
WITH warehouse_tags AS (
    SELECT
        object_name AS warehouse_name,
        tag_value AS cost_center
    FROM SNOWFLAKE.ACCOUNT_USAGE.TAG_REFERENCES
    WHERE domain = 'WAREHOUSE'
        AND tag_name = '{COST_CENTER_TAG}'
        AND deleted IS NULL
)
SELECT
    TO_DATE(wmh.start_time) AS usage_date,
    COALESCE(wt.cost_center, 'SIN_TAG') AS cost_center,
    SUM(wmh.credits_used) AS daily_credits
FROM SNOWFLAKE.ACCOUNT_USAGE.WAREHOUSE_METERING_HISTORY wmh
LEFT JOIN warehouse_tags wt
    ON wmh.warehouse_name = wt.warehouse_name
WHERE wmh.start_time >= DATEADD(day, -{M_DAYS}, CURRENT_TIMESTAMP())
GROUP BY 1, 2
ORDER BY usage_date DESC, daily_credits DESC;
"""
run_sql_df(sql)


### 6.4 Identificar objetos sin tags (oportunidad de mejora)


In [ ]:
# 6.4.1 Warehouses sin tag de COST_CENTER
sql = f"""
WITH tagged_warehouses AS (
    SELECT DISTINCT object_name AS warehouse_name
    FROM SNOWFLAKE.ACCOUNT_USAGE.TAG_REFERENCES
    WHERE domain = 'WAREHOUSE'
        AND tag_name = '{COST_CENTER_TAG}'
        AND deleted IS NULL
)
SELECT
    w.warehouse_name,
    w.warehouse_size,
    w.warehouse_type,
    w.created,
    SUM(wmh.credits_used) AS total_credits_30d
FROM SNOWFLAKE.ACCOUNT_USAGE.WAREHOUSES w
LEFT JOIN SNOWFLAKE.ACCOUNT_USAGE.WAREHOUSE_METERING_HISTORY wmh
    ON w.warehouse_name = wmh.warehouse_name
    AND wmh.start_time >= DATEADD(day, -30, CURRENT_TIMESTAMP())
WHERE w.deleted IS NULL
    AND w.warehouse_name NOT IN (SELECT warehouse_name FROM tagged_warehouses)
GROUP BY 1, 2, 3, 4
ORDER BY total_credits_30d DESC NULLS LAST;
"""
run_sql_df(sql)


---
## 7. Resource Monitors

Los **Resource Monitors** permiten establecer límites de créditos y controlar el gasto.

### Características:
- Monitoreo a nivel de cuenta o warehouse
- Notificaciones al alcanzar umbrales
- Suspensión automática de warehouses
- Ciclos mensuales, semanales o personalizados


### 7.1 Listar Resource Monitors existentes


In [ ]:
# 7.1.1 Ver todos los resource monitors
sql = """
SELECT
    name,
    credit_quota,
    used_credits,
    remaining_credits,
    level,
    frequency,
    start_time,
    end_time,
    notify_at,
    suspend_at,
    suspend_immediately_at,
    created_on,
    owner,
    comment
FROM SNOWFLAKE.ACCOUNT_USAGE.RESOURCE_MONITORS
WHERE deleted IS NULL
ORDER BY name;
"""
run_sql_df(sql)


In [ ]:
# 7.1.2 Resource monitors con uso actual (en tiempo real)
sql = """
SHOW RESOURCE MONITORS;
"""
run_sql_df(sql)


### 7.2 Crear Resource Monitor para un warehouse


In [ ]:
# 7.2.1 Ejemplo: Crear resource monitor mensual
sql_create_rm = f"""
-- Crear resource monitor con quota mensual
CREATE OR REPLACE RESOURCE MONITOR rm_monthly_<DEPARTMENT>
    WITH
        CREDIT_QUOTA = {CREDIT_QUOTA_MONTHLY}
        FREQUENCY = MONTHLY
        START_TIMESTAMP = IMMEDIATELY
        TRIGGERS
            ON 75 PERCENT DO NOTIFY          -- Notificar al 75%
            ON 90 PERCENT DO NOTIFY          -- Notificar al 90%
            ON 100 PERCENT DO SUSPEND        -- Suspender al 100%
            ON 110 PERCENT DO SUSPEND_IMMEDIATE;  -- Suspensión inmediata al 110%

-- Asignar el monitor a un warehouse
ALTER WAREHOUSE <WAREHOUSE_NAME>
    SET RESOURCE_MONITOR = rm_monthly_<DEPARTMENT>;
"""
print("Ejemplo de creación de Resource Monitor:")
print(sql_create_rm)


### 7.3 Crear Resource Monitor a nivel de cuenta


In [ ]:
# 7.3.1 Ejemplo: Resource monitor a nivel de cuenta
sql_account_rm = """
-- Crear resource monitor para toda la cuenta
CREATE OR REPLACE RESOURCE MONITOR rm_account_monthly
    WITH
        CREDIT_QUOTA = 5000
        FREQUENCY = MONTHLY
        START_TIMESTAMP = IMMEDIATELY
        TRIGGERS
            ON 50 PERCENT DO NOTIFY
            ON 75 PERCENT DO NOTIFY
            ON 90 PERCENT DO NOTIFY
            ON 100 PERCENT DO SUSPEND;

-- Asignar a nivel de cuenta (requiere ACCOUNTADMIN)
ALTER ACCOUNT SET RESOURCE_MONITOR = rm_account_monthly;
"""
print("Ejemplo de Resource Monitor a nivel de cuenta:")
print(sql_account_rm)


### 7.4 Warehouses sin Resource Monitor


In [ ]:
# 7.4.1 Identificar warehouses sin resource monitor asignado
sql = """
SELECT
    warehouse_name,
    warehouse_size,
    warehouse_type,
    auto_suspend,
    created,
    'SIN RESOURCE MONITOR' AS status
FROM SNOWFLAKE.ACCOUNT_USAGE.WAREHOUSES
WHERE deleted IS NULL
    AND (resource_monitor IS NULL OR resource_monitor = '')
ORDER BY warehouse_name;
"""
run_sql_df(sql)


---
## 8. Alertas y Detección de Anomalías

Configurar alertas proactivas para monitoreo de costos.


### 8.1 Crear alerta de costos con Snowflake Alerts


In [ ]:
# 8.1.1 Ejemplo: Alerta cuando créditos diarios superan umbral
sql_alert = """
-- Crear alerta para monitoreo de costos
CREATE OR REPLACE ALERT alert_daily_credit_spike
    WAREHOUSE = <ALERT_WAREHOUSE>
    SCHEDULE = 'USING CRON 0 9 * * * America/Mexico_City'  -- Todos los días a las 9am
    IF (EXISTS (
        SELECT 1
        FROM SNOWFLAKE.ACCOUNT_USAGE.WAREHOUSE_METERING_HISTORY
        WHERE TO_DATE(start_time) = CURRENT_DATE() - 1
        GROUP BY TO_DATE(start_time)
        HAVING SUM(credits_used) > 500  -- Umbral de alerta
    ))
    THEN
        CALL SYSTEM$SEND_EMAIL(
            'email_integration',
            'admin@company.com',
            'Alerta: Consumo de créditos elevado',
            'El consumo de créditos del día anterior superó el umbral establecido.'
        );

-- Activar la alerta
ALTER ALERT alert_daily_credit_spike RESUME;
"""
print("Ejemplo de creación de Alerta de costos:")
print(sql_alert)


### 8.2 Query para detectar anomalías de costos


In [ ]:
# 8.2.1 Detectar días con consumo anormal (> 2 desviaciones estándar)
sql = f"""
WITH daily_credits AS (
    SELECT
        TO_DATE(start_time) AS usage_date,
        SUM(credits_used) AS daily_credits
    FROM SNOWFLAKE.ACCOUNT_USAGE.WAREHOUSE_METERING_HISTORY
    WHERE start_time >= DATEADD(day, -{M_DAYS}, CURRENT_TIMESTAMP())
    GROUP BY 1
),
stats AS (
    SELECT
        AVG(daily_credits) AS avg_credits,
        STDDEV(daily_credits) AS stddev_credits
    FROM daily_credits
)
SELECT
    dc.usage_date,
    dc.daily_credits,
    s.avg_credits,
    dc.daily_credits - s.avg_credits AS deviation,
    (dc.daily_credits - s.avg_credits) / NULLIF(s.stddev_credits, 0) AS z_score,
    CASE
        WHEN (dc.daily_credits - s.avg_credits) / NULLIF(s.stddev_credits, 0) > 2 THEN 'ANOMALIA ALTA'
        WHEN (dc.daily_credits - s.avg_credits) / NULLIF(s.stddev_credits, 0) < -2 THEN 'ANOMALIA BAJA'
        ELSE 'NORMAL'
    END AS status
FROM daily_credits dc
CROSS JOIN stats s
ORDER BY dc.usage_date DESC;
"""
run_sql_df(sql)


---
## 9. Buenas Prácticas - Resumen

### Tags:
1. Crear una base de datos centralizada para governance
2. Usar `ALLOWED_VALUES` para tags con valores predefinidos
3. Etiquetar TODOS los warehouses con centro de costo y ambiente
4. Revisar periódicamente objetos sin tags
5. Documentar el propósito de cada tag

### Resource Monitors:
1. Crear resource monitor a nivel de cuenta como respaldo
2. Asignar monitores específicos a warehouses de alto consumo
3. Configurar múltiples umbrales de notificación (50%, 75%, 90%)
4. Usar `SUSPEND` en lugar de `SUSPEND_IMMEDIATE` cuando sea posible
5. Revisar y ajustar quotas mensualmente

### Monitoreo:
1. Crear alertas para anomalías de costos
2. Generar reportes semanales por centro de costo
3. Identificar tendencias de crecimiento de costos
4. Comparar costos DEV vs PROD regularmente


---
### 10. Notas Finales
- Requiere rol `ACCOUNTADMIN` o permisos específicos para crear tags y resource monitors.
- Las vistas de `ACCOUNT_USAGE` tienen una latencia de hasta 45 minutos.
- Considera crear un dashboard con estas queries para monitoreo continuo.
- Revisa la [documentación oficial de Tags](https://docs.snowflake.com/en/user-guide/object-tagging) y [Resource Monitors](https://docs.snowflake.com/en/user-guide/resource-monitors).
